In [ ]:
import xarray as xr
import pystac_client
import planetary_computer

from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)
client

In [ ]:
path = 'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-1440x721.zarr'
era5_ds = xr.open_dataset(path, engine='zarr').chunk('auto')
era5_temperature = era5_ds['2m_temperature']
era5_precipitation = era5_ds['total_precipitation_6hr']

In [ ]:
era5_precipitation

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("cil-gdpcir-cc-by")

summary = collection.summaries.to_dict()
available_keys = list(summary.keys())
print(available_keys)

## What physical variables are stored in the dataset?

In [ ]:
summary['cmip6:variable']

## What GCMs are used in the dataset?

In [ ]:
summary['cmip6:source_id']

##  What are the SSPs in the dataset?

In [ ]:
summary['cmip6:experiment_id']

## How do we use this information to actually get the data?

In [ ]:
search = catalog.search(
    collections=['cil-gdpcir-cc-by'],
    query={'cmip6:experiment_id': {'eq': 'historical'}}
)
historic_data = search.item_collection()
historic_data

In [ ]:
search = catalog.search(
    collections=['cil-gdpcir-cc-by'],
    query={'cmip6:source_id': {'eq': 'GFDL-ESM4'}}
)
gfdl_data = search.item_collection()
gfdl_data